In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.ensemble.weight_boosting import AdaBoostClassifier
# from sklearn.linear_model.ridge import RidgeClassifierCVclear
# from sklearn.linear_model.ridge import RidgeClassifier
# from sklearn.ensemble.bagging import BaggingClassifier
# from sklearn.ensemble.forest import ExtraTreesClassifier
# from sklearn.ensemble.forest import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from xgboost.sklearn import XGBClassifier 


# survival	Survival	0 = No, 1 = Yes
# pclass	Ticket class	1 = 1st, 2 = 2nd, 3 = 3rd
# sex	Sex	
# Age	Age in years	
# sibsp	# of siblings / spouses aboard the Titanic	
# parch	# of parents / children aboard the Titanic	
# ticket	Ticket number	
# fare	Passenger fare	
# cabin	Cabin number	
# embarked	Port of Embarkation	C = Cherbourg, Q = Queenstown, S = Southampton

/home/ds_magico/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.gradient_boosting module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/ds_magico/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.weight_boosting module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
train = pd.read_csv('./train.csv')
test =  pd.read_csv('./test.csv')

In [3]:
data = [train, test]
df_1 = pd.concat(data)
df_1 = df_1.reset_index()
df_1 = df_1.drop(columns='index')

In [4]:
corr1 = df_1.corr()

# - Clean Data

In [5]:
def clean(df):
    
    df = pd.DataFrame(df)
    df.Embarked = [ 0 if i == 'S' else 1 if i == 'C' else 2 for i in df.Embarked]
    df[df.Embarked.isna()] = int(df.Embarked.mean())
    df[df.Fare.isna()] = df.Fare.mean()
    df.Fare = [np.intc(float(str(round(value, 5)).replace('.', ''))) for value in df.Fare]
    df.Sex = [1 if i == "male" else 0 for i in df.Sex]
    
    return df
    
df_2 = clean(df_1)
test_1 = clean(test)

# - Cabin Columns

In [6]:
def clean_cabin(df, drop, na):
    df = pd.DataFrame(df)
    
    if (na == 1):
        df_cabin = df[~df.Cabin.isna()]
        df_cabin.Cabin = [ord(i[0]) - 65 if type(i) != float else int(i) - 28 for i in df_cabin.Cabin]
        df_cabin.Cabin = [ 0 if i in [0, 1, 2] else 1 for i in df_cabin.Cabin]
    else:
        df_cabin = df[df.Cabin.isna()]

    y = df_cabin.Cabin
    x = df_cabin.drop(columns=drop)
    
    return [x, y]
    
def predict_cabin(df, test=0):
    df = pd.DataFrame(df)
    
    # Trein DataSet -> pd.read_csv('./train.csv')
    # Cleaning DataSet
    drop_train = ['PassengerId', 'Survived', 'Ticket', 'Age', 'Name', 'Cabin']
    x_train, y_train = clean_cabin(df, drop_train, 1)  
    
    # Predict train DataSet
    x_train_aux, x_test_aux, y_train_aux, y_test_aux = train_test_split(x_train, y_train, test_size=0.30, random_state=337)
    clf = GradientBoostingClassifier()
    clf.fit(x_train_aux, y_train_aux)
    y_pred = clf.predict(x_test_aux)

    # Predict test DataSet
    x_df_train, y_df_train = clean_cabin(df, drop_train, 0)  
    df_train = clf.predict(x_df_train) 
    
    # Predict final DataSet -> pd.read_csv('./test.csv')
    drop_test = ['PassengerId', 'Ticket', 'Age', 'Name', 'Cabin']
    x_df_test, y_df_test = clean_cabin(test, drop_test, 0)
    df_test = clf.predict(x_df_test)
    
    # join DataSet
    df.loc[~df['Cabin'].isna(), 'Cabin'] = y_train
    df.loc[df['Cabin'].isna(), 'Cabin'] = df_train
    
    useless, y_notna = clean_cabin(test, drop_test, 1)
    test.loc[~test['Cabin'].isna(), 'Cabin'] = y_notna
    test.loc[test['Cabin'].isna(), 'Cabin'] = df_test

    return [df, test]
    
df_3, test_2 = predict_cabin(df_2, test_1)

/home/ds_magico/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/home/ds_magico/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


# - AGE Column

### Data cleaning AGE

In [ ]:
# (df_1.Name.str.contains('Mrs'))

In [ ]:
Miss = [int(y) for y in df_1[(df_1.Name.str.contains('Miss'))].Age.describe()[2:8]]
Mrs = [int(y) for y in df_1[(df_1.Name.str.contains('Mrs'))].Age.describe()[2:8]]
Mr = [int(y) for y in df_1[(df_1.Name.str.contains('Mr'))].Age.describe()[2:8]]
Master = [int(y) for y in df_1[(df_1.Name.str.contains('Master'))].Age.describe()[2:8]]
Outer = [int(y) for y in df_1[~(df_1.Name.str.contains('Miss|Mrs|Master|Mr'))].Age.describe()[2:8]]

def filterAge(title):
    describe = []
    if('Master' in title):
        describe = Master
        li_title = [1]
    elif('Mrs'  in title):
        describe = Mrs
        li_title = [5]
    elif ('Mr' in title):
        describe = Mr
        li_title = [2]
    elif('Miss' in title):
        describe = Miss
        li_title = [3]
    else :
        describe = Outer
        li_title = [4]
    return [describe, li_title]

In [ ]:
age = pd.DataFrame([str(x)[1:-1][1:-1].split(',') for x in df_2.Name.apply(filterAge)])
age.columns = ['Std', 'Min', 'Per_25', 'Per_50', 'Pre_75', 'Max', 'Title']
age.Max = [i[:-1] for i in age.Max]
age.Title = [i[-1:] for i in age.Title]

In [ ]:
def int_age(a):
    return([np.intc(i) for i in a])
    
age = age.apply(int_age)

In [ ]:
df_3 = df_2.join(age)
df_3.Age = [ np.intc(i) if type(i) == int else i for i in df_3.Age]

In [ ]:
df_3

### Predict Age Train/Test

In [ ]:
# Organizando modelos de treino e teste
dfAgeTrain = df_3[df_3.Age.notna()]    # DataFrame que possui idades
dfAgeTest = df_3[df_3.Age.isna()]      # DataFrame que não possui idades

# DataFrame de Treinamento
y_ageTrain = dfAgeTrain['Age']

# target
y_ageTrain = pd.Series([np.intc(value) for value in y_ageTrain])
# Data
x_ageTrain = dfAgeTrain.drop(columns=['PassengerId', 'Name', 'Age', 'Survived', 'Ticket'])

# DataFrame a ser previsto
x_ageTest = dfAgeTest.drop(columns=['PassengerId', 'Name', 'Age', 'Survived', 'Ticket'])

x_train_ageTrain, x_test_ageTrain, y_train_ageTrain, y_test_ageTrain = train_test_split(x_ageTrain, y_ageTrain, test_size=0.3, random_state=337)

In [ ]:
x_train_ageTrain

In [ ]:
clf_age = GradientBoostingRegressor()
clf_age.fit(x_train_ageTrain, y_train_ageTrain)

In [ ]:
y_pred = clf_age.predict(x_test_ageTrain)

In [ ]:
y_pred = [np.intc(value) for value in np.array(y_pred)]
dic = {"Real_age": np.array(y_test_ageTrain), 
       "Pred_age": y_pred
      }

pred = pd.DataFrame(dic)

pred['Erro'] = pred['Real_age'] - pred['Pred_age']

In [ ]:
# Accuracy
pred['Erro'].describe()

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 7))

sns.boxplot(ax=axes[0][0], x='Real_age', data=pred)
sns.boxplot(ax=axes[1][0], x='Pred_age', data=pred)
sns.distplot(pred['Real_age'], ax=axes[0][1], bins=25)
sns.distplot(pred['Pred_age'], ax=axes[1][1], bins=25)

### Predict Age Real

In [ ]:
model = clf_age.predict(x_ageTest)
model = [np.intc(x) for x in model]
dfAgeTest.Age = model

# Survived

In [ ]:
df_4 = pd.concat([dfAgeTest, dfAgeTrain]).sort_values(by='PassengerId')
df_5 = df_4 

df_4.Age = df_4.Age.astype(int) #[np.intc(i) for i in df_4.Age]

### col family_sex


### col family_size
# df_4['Family_size'] = [ i for i in df_4.SibSp + df_4.Parch]


### col sex
# sex = [[1, 0] if 1 == i else [0, 1] for i in df_4.Sex]
# sex = pd.DataFrame(sex)
# sex.columns = ['Male', 'Female']
# df_4 = df_4.join(sex)


### col names_num
# df_4['Names_num'] = [len(i.split()) for i in df_4.Name]


### col Age
# df_4.loc[ df_4['Age'] <= 23, 'Age'] = 0
# df_4.loc[(df_4['Age'] > 23) & (df_4['Age'] <= 34), 'Age'] = 1
# # df_4.loc[(df_4['Age'] > 28) & (df_4['Age'] <= 37), 'Age'] = 2
# # df_4.loc[(df_4['Age'] > 31) & (df_4['Age'] <= 41), 'Age'] = 3
# df_4.loc[ df_4['Age'] > 34, 'Age'] = 2;

### col Alone, max 2 
# 0 if i < 1 else 1 if i == 1 else 2
df_4['Alone'] = [0 if i == 0 else 1 for i in df_4.SibSp + df_4.Parch]


### col X
df_4.x = (df_4.x - 65)
df_4.x = [0 if i in [5, 4] else 1 for i in df_4.x]

In [ ]:
(df_4.groupby('Age').sum() / df_4.groupby('Age').count() * 100).Survived

In [ ]:
df_4.Age.describe()

In [ ]:
df_4 = df_4.iloc[:, :12].join(df_4.iloc[:, -2:])

df_4 = df_4[~df_4.Survived.isna()].drop(columns=['Ticket', 'PassengerId', 'Name', 'Title', 'SibSp', 'Parch'])
df_4.Survived = df_4.Survived.astype(int)
# df_4.Survived = [ np.intc(i) for i in df_4.Survived]

y = df_4.Survived
x = df_4.drop(columns='Survived')

colormap = plt.cm.RdBu
plt.figure(figsize=(17,17))
plt.title('Pearson Correlation of Features', y=1.05, size=15)
sns.heatmap(df_4.astype(float).corr(),linewidths=0.1,vmax=1.0, 
            square=True, cmap=colormap, linecolor='white', annot=True)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=337)

# RandomForestClassifier
# GaussianProcessClassifier
# GradientBoostingClassifier
# AdaBoostClassifier
# RidgeClassifierCV
# RidgeClassifier
# BaggingClassifier
# ExtraTreesClassifier
# RandomForestClassifier
# BernoulliNB
# LinearDiscriminantAnalysis
# XGBClassifier 

clf = XGBClassifier(
        learning_rate = 0.15,
         n_estimators= 2000,
         max_depth= 1,
         min_child_weight= 1,
         #gamma=1,
         gamma=0.9,                        
         subsample=0.79,
         colsample_bytree=0.8,
         objective= 'binary:logistic',
         nthread= -1,
         scale_pos_weight=1
      )
clf.fit(x_train, y_train)

In [ ]:
y_pred = clf.predict(x_test)
accuracy_score(y_test, y_pred) * 100

In [ ]:
test

In [ ]:
# test[test.Fare.isna()] = test.Fare.mean()
df_test = test[test.Fare.isna()]
# df_test = df_test.drop(columns=['Cabin', 'Ticket', 'Name', 'PassengerId', 'Age'])
# model_cabin = clf_cabin_x.predict(df_test)
# df_cabin_2 = df_1[df_1.Cabin.isna()]
# df_cabin_2['x'] = model_cabin

In [ ]:
df_5 = df_5.iloc[:, :12].join(df_4.iloc[:, -2:])
df_5 = df_5[df_5.Survived.isna()].drop(columns=['Ticket', 'PassengerId', 'Name', 'Title', 'SibSp', 'Parch', 'Survived'])
# df_4.Survived = [ np.intc(i) for i in df_4.Survived]

In [ ]:
# submission = pd.DataFrame({
#         "PassengerId": test["PassengerId"],
#         "Survived": pred_real
#     })
# submission.to_csv('./submission.csv', index=False)